In [3]:
import sys
import json
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                             QHBoxLayout, QPushButton, QLabel, QLineEdit, 
                             QMessageBox, QFileDialog, QGroupBox, QFormLayout,
                             QCheckBox, QTabWidget, QFrame, QTextEdit, QSplitter,
                             QTableWidget, QTableWidgetItem, QHeaderView, QProgressBar,
                             QDialog, QScrollArea, QGridLayout, QSizePolicy, QSlider)
from PyQt5.QtCore import Qt, QSettings, QTimer, QMargins, QPropertyAnimation, QEasingCurve
from PyQt5.QtGui import QFont, QPalette, QColor, QIcon, QPixmap, QPainter
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.dates as mdates
from matplotlib import cm
from scipy.interpolate import interp1d

class WaterDropIcon:
    @staticmethod
    def create_icon():
        pixmap = QtGui.QPixmap(32, 32)
        pixmap.fill(Qt.transparent)
        painter = QtGui.QPainter(pixmap)
        painter.setRenderHint(QtGui.QPainter.Antialiasing)
        
        gradient = QtGui.QLinearGradient(8, 4, 24, 24)
        gradient.setColorAt(0, QtGui.QColor(0, 150, 255))
        gradient.setColorAt(1, QtGui.QColor(0, 100, 200))
        
        painter.setBrush(QtGui.QBrush(gradient))
        painter.setPen(QtGui.QPen(QtGui.QColor(0, 80, 180), 1.5))
        painter.drawEllipse(8, 4, 16, 20)
        painter.drawPolygon(QtGui.QPolygon([
            QtCore.QPoint(16, 24),
            QtCore.QPoint(12, 30),
            QtCore.QPoint(20, 30)
        ]))
        
        painter.setBrush(QtGui.QBrush(QtGui.QColor(255, 255, 255, 150)))
        painter.setPen(Qt.NoPen)
        painter.drawEllipse(12, 8, 6, 8)
        
        painter.end()
        return QtGui.QIcon(pixmap)

class AnimatedButton(QPushButton):
    def __init__(self, text, parent=None, enable_animation=True):
        super().__init__(text, parent)
        self.enable_animation = enable_animation
        if self.enable_animation:
            self._animation = QPropertyAnimation(self, b"geometry")
            self._animation.setDuration(200)
            self._animation.setEasingCurve(QEasingCurve.OutCubic)
        
    def mousePressEvent(self, event):
        if self.enable_animation:
            self.perform_animation()
        super().mousePressEvent(event)
        
    def perform_animation(self):
        if hasattr(self, '_animation'):
            original_geometry = self.geometry()
            self._animation.setStartValue(original_geometry)
            self._animation.setEndValue(original_geometry.adjusted(2, 2, -2, -2))
            self._animation.start()

class LoginWindow(QDialog):
    def __init__(self, main_app):
        super().__init__()
        self.main_app = main_app
        self.init_ui()
        
    def init_ui(self):
        self.setWindowTitle("Авторизация - Прогноз потребления воды")
        self.setFixedSize(400, 350)
        self.setWindowIcon(WaterDropIcon.create_icon())
        
        self.setStyleSheet("""
            QWidget {
                background: qlineargradient(x1: 0, y1: 0, x2: 1, y2: 1,
                                         stop: 0 #667eea, stop: 1 #764ba2);
                color: white;
            }
            QLabel {
                color: white;
                background: transparent;
            }
            QGroupBox {
                color: white;
                font-weight: bold;
                border: 2px solid rgba(255,255,255,0.3);
                border-radius: 8px;
                margin-top: 10px;
                padding-top: 10px;
                background: rgba(255,255,255,0.1);
            }
            QGroupBox::title {
                subcontrol-origin: margin;
                left: 10px;
                padding: 0 5px 0 5px;
                color: white;
            }
            QLineEdit {
                background: rgba(255,255,255,0.9);
                border: 1px solid rgba(255,255,255,0.3);
                border-radius: 4px;
                padding: 8px;
                color: #2c3e50;
                font-size: 10pt;
            }
            QLineEdit:focus {
                border: 1px solid #3498db;
                background: white;
            }
            QPushButton {
                background: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1,
                                         stop: 0 #3498db, stop: 1 #2980b9);
                border: none;
                border-radius: 4px;
                color: white;
                padding: 10px;
                font-weight: bold;
                font-size: 10pt;
            }
            QPushButton:hover {
                background: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1,
                                         stop: 0 #3cb0fd, stop: 1 #3498db);
            }
            QPushButton:pressed {
                background: #2980b9;
            }
            QCheckBox {
                color: white;
                background: transparent;
            }
            QCheckBox::indicator {
                width: 16px;
                height: 16px;
                border: 2px solid white;
                border-radius: 3px;
                background: transparent;
            }
            QCheckBox::indicator:checked {
                background: #3498db;
                border: 2px solid #3498db;
            }
        """)
        
        layout = QVBoxLayout()
        layout.setAlignment(Qt.AlignCenter)
        layout.setSpacing(20)
        
        title = QLabel("Прогноз потребления воды")
        title.setAlignment(Qt.AlignCenter)
        title_font = QFont()
        title_font.setPointSize(18)
        title_font.setBold(True)
        title.setFont(title_font)
        layout.addWidget(title)
        
        icon_label = QLabel()
        icon_pixmap = WaterDropIcon.create_icon().pixmap(80, 80)
        icon_label.setPixmap(icon_pixmap)
        icon_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(icon_label)
        
        form_group = QGroupBox("Вход в систему")
        form_layout = QFormLayout()
        form_layout.setVerticalSpacing(15)
        
        self.username_edit = QLineEdit()
        self.username_edit.setPlaceholderText("Введите логин")
        self.password_edit = QLineEdit()
        self.password_edit.setPlaceholderText("Введите пароль")
        self.password_edit.setEchoMode(QLineEdit.Password)
        self.remember_check = QCheckBox("Запомнить меня")
        
        form_layout.addRow("Логин:", self.username_edit)
        form_layout.addRow("Пароль:", self.password_edit)
        form_layout.addRow("", self.remember_check)
        
        form_group.setLayout(form_layout)
        layout.addWidget(form_group)
        
        button_layout = QHBoxLayout()
        self.login_btn = AnimatedButton("Войти")
        self.register_btn = AnimatedButton("Регистрация")
        
        self.login_btn.clicked.connect(self.login)
        self.register_btn.clicked.connect(self.register)
        
        button_layout.addWidget(self.login_btn)
        button_layout.addWidget(self.register_btn)
        layout.addLayout(button_layout)
        
        self.setLayout(layout)
        self.load_saved_credentials()
        
    def load_saved_credentials(self):
        settings = QSettings("WaterCompany", "WaterForecast")
        username = settings.value("username", "")
        password = settings.value("password", "")
        remember = settings.value("remember", False, type=bool)
        
        if remember and username:
            self.username_edit.setText(username)
            self.password_edit.setText(password)
            self.remember_check.setChecked(True)
            
    def save_credentials(self):
        settings = QSettings("WaterCompany", "WaterForecast")
        if self.remember_check.isChecked():
            settings.setValue("username", self.username_edit.text())
            settings.setValue("password", self.password_edit.text())
            settings.setValue("remember", True)
        else:
            settings.remove("username")
            settings.remove("password")
            settings.setValue("remember", False)
            
    def login(self):
        username = self.username_edit.text()
        password = self.password_edit.text()
        
        if not username or not password:
            QMessageBox.warning(self, "Ошибка", "Введите логин и пароль")
            return
            
        if self.main_app.authenticate_user(username, password):
            self.save_credentials()
            self.main_app.current_user = username
            self.main_app.show_main_window()
            self.close()
        else:
            QMessageBox.warning(self, "Ошибка", "Неверный логин или пароль")
            
    def register(self):
        username = self.username_edit.text()
        password = self.password_edit.text()
        
        if not username or not password:
            QMessageBox.warning(self, "Ошибка", "Введите логин и пароль")
            return
            
        if self.main_app.register_user(username, password):
            QMessageBox.information(self, "Успех", "Пользователь зарегистрирован")
            self.save_credentials()
            self.main_app.current_user = username
            self.main_app.show_main_window()
            self.close()
        else:
            QMessageBox.warning(self, "Ошибка", "Пользователь уже существует")

class ThreeDVisualizationDialog(QDialog):
    def __init__(self, parent=None, viz_type=None, data=None):
        super().__init__(parent)
        self.viz_type = viz_type
        self.data = data
        self.setWindowTitle(f"3D Визуализация - {viz_type}")
        self.setGeometry(100, 50, 1200, 900)
        self.setWindowIcon(WaterDropIcon.create_icon())
        self.init_ui()
        
    def init_ui(self):
        layout = QVBoxLayout()
        layout.setContentsMargins(10, 10, 10, 10)
        layout.setSpacing(10)
        
        # Заголовок и кнопки управления
        header_layout = QHBoxLayout()
        
        title = QLabel(f"3D Визуализация: {self.viz_type}")
        title_font = QFont()
        title_font.setPointSize(14)
        title_font.setBold(True)
        title.setFont(title_font)
        title.setStyleSheet("color: #2c3e50;")
        header_layout.addWidget(title)
        
        # Кнопки управления
        button_layout = QHBoxLayout()
        
        self.zoom_in_btn = AnimatedButton("➕")
        self.zoom_in_btn.setToolTip("Увеличить")
        self.zoom_in_btn.clicked.connect(self.zoom_in)
        self.zoom_in_btn.setFixedSize(35, 30)
        
        self.zoom_out_btn = AnimatedButton("➖")
        self.zoom_out_btn.setToolTip("Уменьшить")
        self.zoom_out_btn.clicked.connect(self.zoom_out)
        self.zoom_out_btn.setFixedSize(35, 30)
        
        self.zoom_reset_btn = AnimatedButton("🔍")
        self.zoom_reset_btn.setToolTip("Сбросить масштаб")
        self.zoom_reset_btn.clicked.connect(self.zoom_reset)
        self.zoom_reset_btn.setFixedSize(35, 30)
        
        self.rotate_btn = AnimatedButton("🔄")
        self.rotate_btn.setToolTip("Вращать график")
        self.rotate_btn.clicked.connect(self.toggle_rotation)
        self.rotate_btn.setFixedSize(35, 30)
        
        close_btn = AnimatedButton("Закрыть")
        close_btn.clicked.connect(self.close)
        close_btn.setFixedSize(80, 30)
        
        button_layout.addWidget(self.zoom_in_btn)
        button_layout.addWidget(self.zoom_out_btn)
        button_layout.addWidget(self.zoom_reset_btn)
        button_layout.addWidget(self.rotate_btn)
        button_layout.addStretch()
        button_layout.addWidget(close_btn)
        
        header_layout.addLayout(button_layout)
        layout.addLayout(header_layout)
        
        # Область с прокруткой для графика
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        scroll_area.setHorizontalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        scroll_area.setVerticalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        scroll_area.setMinimumSize(1000, 700)
        
        # Контейнер для графика
        self.graph_container = QWidget()
        self.graph_layout = QVBoxLayout(self.graph_container)
        self.graph_layout.setContentsMargins(0, 0, 0, 0)
        
        # Создаем большую фигуру для полноэкранного режима
        self.figure_3d = Figure(figsize=(16, 12), dpi=100)
        self.canvas_3d = FigureCanvas(self.figure_3d)
        self.canvas_3d.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        self.canvas_3d.setMinimumSize(1000, 700)
        
        self.graph_layout.addWidget(self.canvas_3d)
        scroll_area.setWidget(self.graph_container)
        layout.addWidget(scroll_area)
        
        # Информационная панель
        info_layout = QHBoxLayout()
        
        self.zoom_label = QLabel("Масштаб: 100%")
        self.zoom_label.setStyleSheet("color: #7f8c8d; font-size: 9pt;")
        
        self.rotation_label = QLabel("Вращение: выкл")
        self.rotation_label.setStyleSheet("color: #7f8c8d; font-size: 9pt;")
        
        info_layout.addWidget(self.zoom_label)
        info_layout.addStretch()
        info_layout.addWidget(self.rotation_label)
        
        layout.addLayout(info_layout)
        
        self.setLayout(layout)
        
        # Инициализация переменных
        self.current_zoom = 100
        self.rotation_running = False
        self.rotation_angle = 0
        self.current_3d_ax = None
        
        # Создаем график
        self.create_3d_visualization()
        
    def create_3d_visualization(self):
        """Создает 3D визуализацию"""
        try:
            dates = self.data['dates']
            consumption = self.data['consumption']
            confidence_lower = self.data.get('confidence_lower')
            confidence_upper = self.data.get('confidence_upper')
            
            # Очищаем предыдущий график
            self.figure_3d.clear()
            
            # Создаем выбранный тип визуализации
            if self.viz_type == 'waterfall':
                fig, ax = self.create_3d_waterfall(dates, consumption, confidence_lower, confidence_upper)
            elif self.viz_type == 'surface':
                fig, ax = self.create_3d_surface(dates, consumption)
            elif self.viz_type == 'ribbon':
                fig, ax = self.create_3d_ribbon(dates, consumption, confidence_lower, confidence_upper)
            elif self.viz_type == 'timeseries':
                fig, ax = self.create_3d_timeseries(dates, consumption)
            else:
                return
                
            self.current_3d_ax = ax
            self.canvas_3d.draw()
            
        except Exception as e:
            QMessageBox.critical(self, "Ошибка", f"Ошибка при построении 3D графика: {str(e)}")
    
    def create_3d_waterfall(self, dates, consumption, confidence_lower=None, confidence_upper=None):
        """Создает 3D waterfall график потребления"""
        ax = self.figure_3d.add_subplot(111, projection='3d')
        
        # Преобразуем даты в числовой формат
        dates_num = mdates.date2num(dates)
        
        # Создаем базовые координаты
        x_pos = np.arange(len(dates))
        y_pos = np.zeros(len(dates))
        
        # Высота столбцов - потребление
        z_pos = np.zeros(len(dates))
        dx = dy = 0.8
        dz = consumption
        
        # Нормализуем потребление для цветовой карты
        if consumption.max() - consumption.min() > 0:
            colors = cm.viridis((consumption - consumption.min()) / (consumption.max() - consumption.min()))
        else:
            colors = cm.viridis(np.zeros_like(consumption))
        
        # Создаем 3D столбцы
        ax.bar3d(x_pos, y_pos, z_pos, dx, dy, dz, color=colors, alpha=0.8, shade=True)
        
        # Добавляем доверительные интервалы если есть
        if confidence_lower is not None and confidence_upper is not None:
            for i, (x, lower, upper) in enumerate(zip(x_pos, confidence_lower, confidence_upper)):
                ax.plot([x + dx/2, x + dx/2], [0, 0], [lower, upper], 
                       'r-', linewidth=2, alpha=0.7)
        
        # Настройка осей
        ax.set_xlabel('Временные точки', fontsize=12, labelpad=15)
        ax.set_ylabel('')
        ax.set_zlabel('Потребление воды, тонн', fontsize=12, labelpad=15)
        ax.set_title('3D Визуализация потребления воды\nWaterfall Chart', fontsize=16, pad=30)
        
        # Настраиваем метки на оси X
        step = max(1, len(x_pos) // 8)
        ax.set_xticks(x_pos[::step])
        ax.set_xticklabels([dates[i].strftime('%d.%m') for i in range(len(dates))][::step], 
                          rotation=45, fontsize=10)
        
        # Увеличиваем размер шрифта на осях
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10, pad=8)
        
        # Начальный угол обзора
        ax.view_init(elev=25, azim=45)
        
        # Автоматическое масштабирование
        ax.autoscale(enable=True, axis='both', tight=True)
        
        self.figure_3d.tight_layout()
        return self.figure_3d, ax
    
    def create_3d_surface(self, dates, consumption):
        """Создает 3D поверхность тренда потребления"""
        ax = self.figure_3d.add_subplot(111, projection='3d')
        
        # Создаем сетку для поверхности
        x = np.arange(len(dates))
        y = np.linspace(0, 1, 30)
        
        X, Y = np.meshgrid(x, y)
        
        # Интерполируем потребление для создания плавной поверхности
        try:
            consumption_interp = interp1d(x, consumption, kind='cubic', fill_value='extrapolate')
            Z = consumption_interp(X) * (1 - Y)
        except:
            consumption_interp = interp1d(x, consumption, kind='linear', fill_value='extrapolate')
            Z = consumption_interp(X) * (1 - Y)
        
        # Создаем поверхность
        surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, alpha=0.8, 
                              linewidth=0, antialiased=True)
        
        # Добавляем исходную линию данных
        ax.plot(x, np.zeros_like(x), consumption, 'r-', linewidth=3, label='Фактическое потребление')
        
        # Настройка осей
        ax.set_xlabel('Временные точки', fontsize=12, labelpad=15)
        ax.set_ylabel('Интенсивность тренда', fontsize=12, labelpad=15)
        ax.set_zlabel('Потребление воды, тонн', fontsize=12, labelpad=15)
        ax.set_title('3D Поверхность тренда потребления воды', fontsize=16, pad=30)
        
        # Цветовая шкала
        self.figure_3d.colorbar(surf, ax=ax, shrink=0.6, aspect=20, label='Потребление, тонн')
        
        # Увеличиваем размер шрифта на осях
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10, pad=8)
        
        ax.view_init(elev=20, azim=45)
        ax.legend(fontsize=10, loc='upper left')
        
        # Автоматическое масштабирование
        ax.autoscale(enable=True, axis='both', tight=True)
        
        self.figure_3d.tight_layout()
        return self.figure_3d, ax
    
    def create_3d_ribbon(self, dates, consumption, confidence_lower=None, confidence_upper=None):
        """Создает 3D ribbon график с доверительными интервалами"""
        ax = self.figure_3d.add_subplot(111, projection='3d')
        
        x = np.arange(len(dates))
        
        # Основная линия потребления
        ax.plot(x, consumption, zs=0, zdir='z', color='blue', linewidth=3, label='Потребление')
        
        # Создаем ribbon если есть доверительные интервалы
        if confidence_lower is not None and confidence_upper is not None:
            # Нижняя граница
            ax.plot(x, confidence_lower, zs=0, zdir='z', color='red', linewidth=2, alpha=0.7, label='Нижняя граница')
            # Верхняя граница
            ax.plot(x, confidence_upper, zs=0, zdir='z', color='green', linewidth=2, alpha=0.7, label='Верхняя граница')
            
            # Заполняем область между границами
            for i in range(len(x)-1):
                x_fill = [x[i], x[i], x[i+1], x[i+1]]
                y_fill = [confidence_lower[i], confidence_upper[i], confidence_upper[i+1], confidence_lower[i+1]]
                ax.add_collection3d(plt.fill(x_fill, y_fill, alpha=0.2, color='gray'), zs=0, zdir='z')
        
        # Настройка осей
        ax.set_xlabel('Временные точки', fontsize=12, labelpad=15)
        ax.set_ylabel('Потребление воды, тонн', fontsize=12, labelpad=15)
        ax.set_zlabel('')
        ax.set_title('3D Ribbon график потребления воды', fontsize=16, pad=30)
        
        # Увеличиваем размер шрифта на осях
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10, pad=8)
        
        ax.view_init(elev=15, azim=45)
        ax.legend(fontsize=10, loc='upper left')
        
        # Автоматическое масштабирование
        ax.autoscale(enable=True, axis='both', tight=True)
        
        self.figure_3d.tight_layout()
        return self.figure_3d, ax
    
    def create_3d_timeseries(self, dates, consumption):
        """Создает 3D временной ряд с дополнительными измерениями"""
        ax = self.figure_3d.add_subplot(111, projection='3d')
        
        # Преобразуем даты в числовой формат
        dates_num = mdates.date2num(dates)
        
        # Вычисляем скользящее среднее для третьего измерения
        window_size = min(7, len(consumption) // 4)
        if window_size > 1:
            moving_avg = pd.Series(consumption).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(method='ffill')
        else:
            moving_avg = consumption
        
        # Создаем 3D линию
        ax.plot(dates_num, consumption, moving_avg, 
               linewidth=3, color='purple', alpha=0.8, label='3D временной ряд')
        
        # Добавляем точки данных
        scatter = ax.scatter(dates_num, consumption, moving_avg, 
                  c=consumption, cmap='viridis', s=50, alpha=0.6)
        
        # Настройка осей
        ax.set_xlabel('Дата', fontsize=12, labelpad=15)
        ax.set_ylabel('Потребление, тонн', fontsize=12, labelpad=15)
        ax.set_zlabel(f'Скользящее среднее ({window_size} дней)', fontsize=12, labelpad=15)
        ax.set_title('3D Временной ряд потребления воды', fontsize=16, pad=30)
        
        # Форматирование дат на оси X
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))
        
        # Цветовая шкала для точек
        self.figure_3d.colorbar(scatter, ax=ax, shrink=0.6, aspect=20, label='Потребление, тонн')
        
        # Увеличиваем размер шрифта на осях
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10, pad=8)
        
        ax.view_init(elev=20, azim=45)
        ax.legend(fontsize=10, loc='upper left')
        
        # Автоматическое масштабирование
        ax.autoscale(enable=True, axis='both', tight=True)
        
        self.figure_3d.tight_layout()
        return self.figure_3d, ax
    
    def zoom_in(self):
        """Увеличить масштаб"""
        if self.current_zoom < 200:
            self.current_zoom += 10
            self.update_zoom()
    
    def zoom_out(self):
        """Уменьшить масштаб"""
        if self.current_zoom > 50:
            self.current_zoom -= 10
            self.update_zoom()
    
    def zoom_reset(self):
        """Сбросить масштаб"""
        self.current_zoom = 100
        self.update_zoom()
    
    def update_zoom(self):
        """Обновить масштаб графика"""
        self.zoom_label.setText(f"Масштаб: {self.current_zoom}%")
        # Здесь можно добавить логику изменения размера графика
        self.canvas_3d.draw()
    
    def toggle_rotation(self):
        """Переключить вращение графика"""
        if not self.rotation_running:
            self.start_rotation()
        else:
            self.stop_rotation()
    
    def start_rotation(self):
        """Начать вращение графика"""
        self.rotation_running = True
        self.rotation_label.setText("Вращение: вкл")
        self.rotation_angle = 0
        
        self.rotation_timer = QTimer()
        self.rotation_timer.timeout.connect(self.rotate_3d)
        self.rotation_timer.start(50)
    
    def stop_rotation(self):
        """Остановить вращение графика"""
        self.rotation_running = False
        self.rotation_label.setText("Вращение: выкл")
        
        if hasattr(self, 'rotation_timer'):
            self.rotation_timer.stop()
    
    def rotate_3d(self):
        """Вращать 3D график"""
        if not hasattr(self, 'current_3d_ax') or self.current_3d_ax is None:
            return
            
        self.rotation_angle = (self.rotation_angle + 1) % 360
        
        # Плавное изменение угла обзора
        elevation = 15 + 10 * np.sin(np.radians(self.rotation_angle * 2))
        
        self.current_3d_ax.view_init(elev=elevation, azim=self.rotation_angle)
        self.canvas_3d.draw()

class AnalysisResultsWindow(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Детальный анализ прогноза")
        self.setGeometry(200, 200, 900, 700)
        self.setWindowIcon(WaterDropIcon.create_icon())
        self.is_fullscreen = False
        self.init_ui()
        
    def init_ui(self):
        layout = QVBoxLayout()
        layout.setContentsMargins(15, 15, 15, 15)
        layout.setSpacing(15)
        
        # Заголовок с кнопками управления масштабированием
        header_layout = QHBoxLayout()
        
        title = QLabel("Детальный анализ прогноза потребления воды")
        title.setAlignment(Qt.AlignCenter)
        title_font = QFont()
        title_font.setPointSize(16)
        title_font.setBold(True)
        title.setFont(title_font)
        title.setStyleSheet("color: #2c3e50; margin: 10px;")
        header_layout.addWidget(title)
        
        # Кнопки управления масштабированием
        zoom_buttons_layout = QHBoxLayout()
        
        self.zoom_in_btn = AnimatedButton("➕")
        self.zoom_in_btn.setToolTip("Увеличить масштаб текста")
        self.zoom_in_btn.clicked.connect(self.zoom_in)
        self.zoom_in_btn.setFixedSize(40, 30)
        self.zoom_in_btn.setStyleSheet("""
            QPushButton {
                background-color: #27ae60;
                color: white;
                border-radius: 4px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #219653;
            }
        """)
        
        self.zoom_out_btn = AnimatedButton("➖")
        self.zoom_out_btn.setToolTip("Уменьшить масштаб текста")
        self.zoom_out_btn.clicked.connect(self.zoom_out)
        self.zoom_out_btn.setFixedSize(40, 30)
        self.zoom_out_btn.setStyleSheet("""
            QPushButton {
                background-color: #e67e22;
                color: white;
                border-radius: 4px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #d35400;
            }
        """)
        
        self.zoom_reset_btn = AnimatedButton("🔍")
        self.zoom_reset_btn.setToolTip("Сбросить масштаб")
        self.zoom_reset_btn.clicked.connect(self.zoom_reset)
        self.zoom_reset_btn.setFixedSize(40, 30)
        self.zoom_reset_btn.setStyleSheet("""
            QPushButton {
                background-color: #3498db;
                color: white;
                border-radius: 4px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #2980b9;
            }
        """)
        
        self.fullscreen_btn = AnimatedButton("⛶")
        self.fullscreen_btn.setToolTip("Полноэкранный режим")
        self.fullscreen_btn.clicked.connect(self.toggle_fullscreen)
        self.fullscreen_btn.setFixedSize(40, 30)
        self.fullscreen_btn.setStyleSheet("""
            QPushButton {
                background-color: #9b59b6;
                color: white;
                border-radius: 4px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #8e44ad;
            }
        """)
        
        zoom_buttons_layout.addWidget(self.zoom_in_btn)
        zoom_buttons_layout.addWidget(self.zoom_out_btn)
        zoom_buttons_layout.addWidget(self.zoom_reset_btn)
        zoom_buttons_layout.addWidget(self.fullscreen_btn)
        zoom_buttons_layout.addStretch()
        
        header_layout.addLayout(zoom_buttons_layout)
        layout.addLayout(header_layout)
        
        line = QFrame()
        line.setFrameShape(QFrame.HLine)
        line.setFrameShadow(QFrame.Sunken)
        line.setStyleSheet("background-color: #bdc3c7;")
        layout.addWidget(line)
        
        # Основная область с прокруткой
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        scroll_area.setHorizontalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        scroll_area.setVerticalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        
        self.scroll_widget = QWidget()
        self.scroll_layout = QVBoxLayout(self.scroll_widget)
        self.scroll_layout.setContentsMargins(20, 20, 20, 20)
        self.scroll_layout.setSpacing(20)
        
        # Текущий масштаб
        self.current_zoom = 100
        self.zoom_factor = 1.0
        
        # Статистика потребления
        self.stats_group = QGroupBox("📊 Статистика потребления")
        self.stats_group.setStyleSheet("QGroupBox { font-weight: bold; color: #2c3e50; }")
        stats_layout = QGridLayout()
        stats_layout.setVerticalSpacing(12)
        stats_layout.setHorizontalSpacing(25)
        
        self.stats_labels = {}
        stats_items = [
            ("Период анализа:", "period"),
            ("Начальное потребление:", "start_cons"),
            ("Конечное потребление:", "end_cons"),
            ("Среднее потребление:", "avg_cons"),
            ("Максимальное потребление:", "max_cons"),
            ("Минимальное потребление:", "min_cons"),
            ("Общее потребление:", "total_cons"),
            ("Стандартное отклонение:", "std_dev"),
            ("Коэффициент вариации:", "var_coef")
        ]
        
        for i, (label_text, key) in enumerate(stats_items):
            label = QLabel(label_text)
            label.setStyleSheet("font-size: 10pt;")
            value = QLabel("—")
            value.setStyleSheet("font-weight: bold; color: #2c3e50; font-size: 10pt;")
            value.setWordWrap(True)
            stats_layout.addWidget(label, i, 0)
            stats_layout.addWidget(value, i, 1)
            self.stats_labels[key] = value
        
        self.stats_group.setLayout(stats_layout)
        self.scroll_layout.addWidget(self.stats_group)
        
        # Сезонный анализ
        self.season_group = QGroupBox("🌡️ Сезонный анализ")
        self.season_group.setStyleSheet("QGroupBox { font-weight: bold; color: #2c3e50; }")
        self.season_layout = QGridLayout()
        self.season_layout.setVerticalSpacing(12)
        self.season_layout.setHorizontalSpacing(25)
        self.season_group.setLayout(self.season_layout)
        self.scroll_layout.addWidget(self.season_group)
        
        # Рекомендации
        self.recommendations_group = QGroupBox("💡 Рекомендации по управлению водными ресурсами")
        self.recommendations_group.setStyleSheet("QGroupBox { font-weight: bold; color: #2c3e50; }")
        rec_layout = QVBoxLayout()
        
        self.recommendations_text = QTextEdit()
        self.recommendations_text.setReadOnly(True)
        self.recommendations_text.setMinimumHeight(250)
        self.recommendations_text.setStyleSheet("""
            QTextEdit {
                background-color: #f8f9fa;
                border: 1px solid #dee2e6;
                border-radius: 5px;
                padding: 15px;
                font-size: 10pt;
                line-height: 1.4;
            }
        """)
        rec_layout.addWidget(self.recommendations_text)
        
        self.recommendations_group.setLayout(rec_layout)
        self.scroll_layout.addWidget(self.recommendations_group)
        
        # Прогноз
        self.forecast_group = QGroupBox("🔮 Прогноз на ближайший период")
        self.forecast_group.setStyleSheet("QGroupBox { font-weight: bold; color: #2c3e50; }")
        forecast_layout = QVBoxLayout()
        
        self.forecast_text = QTextEdit()
        self.forecast_text.setReadOnly(True)
        self.forecast_text.setMinimumHeight(180)
        self.forecast_text.setStyleSheet("""
            QTextEdit {
                background-color: #e8f4fd;
                border: 1px solid #b8daff;
                border-radius: 5px;
                padding: 15px;
                font-size: 10pt;
                line-height: 1.4;
            }
        """)
        forecast_layout.addWidget(self.forecast_text)
        
        self.forecast_group.setLayout(forecast_layout)
        self.scroll_layout.addWidget(self.forecast_group)
        
        self.scroll_layout.addStretch()
        
        scroll_area.setWidget(self.scroll_widget)
        layout.addWidget(scroll_area)
        
        # Индикатор масштаба
        self.zoom_label = QLabel(f"Масштаб: {self.current_zoom}%")
        self.zoom_label.setAlignment(Qt.AlignRight)
        self.zoom_label.setStyleSheet("color: #7f8c8d; font-size: 9pt; padding: 5px;")
        layout.addWidget(self.zoom_label)
        
        # Кнопки внизу
        button_layout = QHBoxLayout()
        
        close_btn = AnimatedButton("Закрыть")
        close_btn.clicked.connect(self.close)
        close_btn.setStyleSheet("""
            QPushButton {
                background-color: #95a5a6;
                color: white;
                padding: 8px 16px;
                border-radius: 4px;
                font-weight: bold;
                min-width: 100px;
            }
            QPushButton:hover {
                background-color: #7f8c8d;
            }
        """)
        
        button_layout.addStretch()
        button_layout.addWidget(close_btn)
        layout.addLayout(button_layout)
        
        self.setLayout(layout)
        
        # Устанавливаем начальный размер шрифта
        self.update_font_sizes()

    def zoom_in(self):
        """Увеличить масштаб"""
        if self.current_zoom < 200:
            self.current_zoom += 10
            self.zoom_factor = self.current_zoom / 100.0
            self.update_font_sizes()
            self.update_zoom_label()

    def zoom_out(self):
        """Уменьшить масштаб"""
        if self.current_zoom > 50:
            self.current_zoom -= 10
            self.zoom_factor = self.current_zoom / 100.0
            self.update_font_sizes()
            self.update_zoom_label()

    def zoom_reset(self):
        """Сбросить масштаб"""
        self.current_zoom = 100
        self.zoom_factor = 1.0
        self.update_font_sizes()
        self.update_zoom_label()

    def update_zoom_label(self):
        """Обновить индикатор масштаба"""
        self.zoom_label.setText(f"Масштаб: {self.current_zoom}%")

    def update_font_sizes(self):
        """Обновить размеры шрифтов согласно текущему масштабу"""
        base_size = 10
        
        # Обновляем стили для всех элементов
        style_sheet = f"""
            QGroupBox {{
                font-weight: bold; 
                color: #2c3e50;
                font-size: {base_size * self.zoom_factor}pt;
            }}
            QLabel {{
                font-size: {base_size * self.zoom_factor}pt;
            }}
            QTextEdit {{
                font-size: {base_size * self.zoom_factor}pt;
                line-height: {1.4 * self.zoom_factor};
            }}
        """
        
        # Применяем стили ко всем виджетам
        widgets = [
            self.stats_group, self.season_group, 
            self.recommendations_group, self.forecast_group
        ]
        
        for widget in widgets:
            if hasattr(widget, 'setStyleSheet'):
                widget.setStyleSheet(style_sheet)
        
        # Обновляем конкретные текстовые поля
        text_edit_style = f"""
            QTextEdit {{
                background-color: #f8f9fa;
                border: 1px solid #dee2e6;
                border-radius: 5px;
                padding: {15 * self.zoom_factor}px;
                font-size: {base_size * self.zoom_factor}pt;
                line-height: {1.4 * self.zoom_factor};
            }}
        """
        
        forecast_style = f"""
            QTextEdit {{
                background-color: #e8f4fd;
                border: 1px solid #b8daff;
                border-radius: 5px;
                padding: {15 * self.zoom_factor}px;
                font-size: {base_size * self.zoom_factor}pt;
                line-height: {1.4 * self.zoom_factor};
            }}
        """
        
        self.recommendations_text.setStyleSheet(text_edit_style)
        self.forecast_text.setStyleSheet(forecast_style)

    def toggle_fullscreen(self):
        """Переключить полноэкранный режим - упрощенная версия"""
        if not self.is_fullscreen:
            # Просто переходим в полноэкранный режим
            self.showFullScreen()
            self.is_fullscreen = True
            self.fullscreen_btn.setText("⛶")
            self.fullscreen_btn.setToolTip("Выйти из полноэкранного режима")
        else:
            # Выходим из полноэкранного режима
            self.showNormal()
            self.is_fullscreen = False
            self.fullscreen_btn.setText("⛶")
            self.fullscreen_btn.setToolTip("Полноэкранный режим")

    def keyPressEvent(self, event):
        """Обработка нажатий клавиш - упрощенная версия"""
        if event.key() == Qt.Key_Escape and self.is_fullscreen:
            self.toggle_fullscreen()
        elif event.key() == Qt.Key_Plus and event.modifiers() & Qt.ControlModifier:
            self.zoom_in()
        elif event.key() == Qt.Key_Minus and event.modifiers() & Qt.ControlModifier:
            self.zoom_out()
        elif event.key() == Qt.Key_0 and event.modifiers() & Qt.ControlModifier:
            self.zoom_reset()
        else:
            super().keyPressEvent(event)

    def wheelEvent(self, event):
        """Обработка прокрутки колесика мыши с зажатым Ctrl для масштабирования"""
        if event.modifiers() & Qt.ControlModifier:
            if event.angleDelta().y() > 0:
                self.zoom_in()
            else:
                self.zoom_out()
            event.accept()
        else:
            super().wheelEvent(event)

    def showEvent(self, event):
        """Анимация при показе окна - упрощенная версия"""
        self.animation = QPropertyAnimation(self, b"windowOpacity")
        self.animation.setDuration(300)
        self.animation.setStartValue(0)
        self.animation.setEndValue(1)
        self.animation.start()
        super().showEvent(event)
        
    def set_analysis_data(self, data_stats, seasonal_data, recommendations, forecast):
        """Установка данных анализа"""
        # Обновляем статистику
        self.stats_labels['period'].setText(data_stats.get('period', '—'))
        self.stats_labels['start_cons'].setText(data_stats.get('start_cons', '—'))
        self.stats_labels['end_cons'].setText(data_stats.get('end_cons', '—'))
        self.stats_labels['avg_cons'].setText(data_stats.get('avg_cons', '—'))
        self.stats_labels['max_cons'].setText(data_stats.get('max_cons', '—'))
        self.stats_labels['min_cons'].setText(data_stats.get('min_cons', '—'))
        self.stats_labels['total_cons'].setText(data_stats.get('total_cons', '—'))
        self.stats_labels['std_dev'].setText(data_stats.get('std_dev', '—'))
        self.stats_labels['var_coef'].setText(data_stats.get('var_coef', '—'))
        
        # Обновляем сезонный анализ
        if seasonal_data:
            # Очищаем предыдущие данные
            for i in reversed(range(self.season_layout.count())):
                widget = self.season_layout.itemAt(i).widget()
                if widget:
                    widget.deleteLater()
            
            # Добавляем новые данные
            for i, (season, data) in enumerate(seasonal_data.items()):
                season_label = QLabel(season)
                season_label.setStyleSheet("font-size: 10pt;")
                season_value = QLabel(data)
                season_value.setStyleSheet("font-weight: bold; font-size: 10pt;")
                season_value.setWordWrap(True)
                self.season_layout.addWidget(season_label, i, 0)
                self.season_layout.addWidget(season_value, i, 1)
        else:
            self.season_group.hide()
        
        # Обновляем рекомендации и прогноз
        self.recommendations_text.setText(recommendations)
        self.forecast_text.setText(forecast)

class Water3DVisualizer:
    """Класс для создания 3D визуализаций потребления воды"""
    
    @staticmethod
    def create_3d_waterfall(dates, consumption, confidence_lower=None, confidence_upper=None):
        """Создает 3D waterfall график потребления"""
        fig = Figure(figsize=(16, 12))
        ax = fig.add_subplot(111, projection='3d')
        
        # Преобразуем даты в числовой формат
        dates_num = mdates.date2num(dates)
        
        # Создаем базовые координаты
        x_pos = np.arange(len(dates))
        y_pos = np.zeros(len(dates))
        
        # Высота столбцов - потребление
        z_pos = np.zeros(len(dates))
        dx = dy = 0.8
        dz = consumption
        
        # Цвета в зависимости от уровня потребления
        colors = cm.viridis((consumption - consumption.min()) / (consumption.max() - consumption.min()))
        
        # Создаем 3D столбцы
        ax.bar3d(x_pos, y_pos, z_pos, dx, dy, dz, color=colors, alpha=0.8, shade=True)
        
        # Добавляем доверительные интервалы если есть
        if confidence_lower is not None and confidence_upper is not None:
            for i, (x, lower, upper) in enumerate(zip(x_pos, confidence_lower, confidence_upper)):
                ax.plot([x + dx/2, x + dx/2], [0, 0], [lower, upper], 
                       'r-', linewidth=2, alpha=0.7)
        
        # Настройка осей
        ax.set_xlabel('Временные точки', fontsize=12, labelpad=15)
        ax.set_ylabel('')
        ax.set_zlabel('Потребление воды, тонн', fontsize=12, labelpad=15)
        ax.set_title('3D Визуализация потребления воды\nWaterfall Chart', fontsize=16, pad=30)
        
        # Настраиваем метки на оси X
        ax.set_xticks(x_pos[::max(1, len(x_pos)//10)])  # Показываем каждую 10-ю метку
        ax.set_xticklabels([dates[i].strftime('%d.%m') for i in range(len(dates))][::max(1, len(x_pos)//10)], 
                          rotation=45)
        
        # Увеличиваем размер шрифта на осях
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10, pad=8)
        
        # Начальный угол обзора
        ax.view_init(elev=25, azim=45)
        
        return fig, ax
    
    @staticmethod
    def create_3d_surface(dates, consumption):
        """Создает 3D поверхность тренда потребления"""
        fig = Figure(figsize=(16, 12))
        ax = fig.add_subplot(111, projection='3d')
        
        # Создаем сетку для поверхности
        x = np.arange(len(dates))
        y = np.linspace(0, 1, 50)  # Второе измерение для создания поверхности
        
        X, Y = np.meshgrid(x, y)
        
        # Интерполируем потребление для создания плавной поверхности
        consumption_interp = interp1d(x, consumption, kind='cubic', fill_value='extrapolate')
        Z = consumption_interp(X) * (1 - Y)  # Создаем наклонную поверхность
        
        # Создаем поверхность
        surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, alpha=0.8, 
                              linewidth=0, antialiased=True)
        
        # Добавляем исходную линию данных
        ax.plot(x, np.zeros_like(x), consumption, 'r-', linewidth=3, label='Фактическое потребление')
        
        # Настройка осей
        ax.set_xlabel('Временные точки', fontsize=12, labelpad=15)
        ax.set_ylabel('Интенсивность тренда', fontsize=12, labelpad=15)
        ax.set_zlabel('Потребление воды, тонн', fontsize=12, labelpad=15)
        ax.set_title('3D Поверхность тренда потребления воды', fontsize=16, pad=30)
        
        # Цветовая шкала
        fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5, label='Потребление, тонн')
        
        # Увеличиваем размер шрифта на осях
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10, pad=8)
        
        ax.view_init(elev=20, azim=45)
        ax.legend(fontsize=10)
        
        return fig, ax
    
    @staticmethod
    def create_3d_ribbon(dates, consumption, confidence_lower=None, confidence_upper=None):
        """Создает 3D ribbon график с доверительными интервалами"""
        fig = Figure(figsize=(16, 12))
        ax = fig.add_subplot(111, projection='3d')
        
        x = np.arange(len(dates))
        
        # Основная линия потребления
        ax.plot(x, consumption, zs=0, zdir='z', color='blue', linewidth=3, label='Потребление')
        
        # Создаем ribbon если есть доверительные интервалы
        if confidence_lower is not None and confidence_upper is not None:
            # Нижняя граница
            ax.plot(x, confidence_lower, zs=0, zdir='z', color='red', linewidth=1, alpha=0.7, label='Нижняя граница')
            # Верхняя граница
            ax.plot(x, confidence_upper, zs=0, zdir='z', color='green', linewidth=1, alpha=0.7, label='Верхняя граница')
            
            # Заполняем область между границами
            for i in range(len(x)-1):
                x_fill = [x[i], x[i], x[i+1], x[i+1]]
                y_fill = [confidence_lower[i], confidence_upper[i], confidence_upper[i+1], confidence_lower[i+1]]
                ax.add_collection3d(plt.fill(x_fill, y_fill, alpha=0.2, color='gray'), zs=0, zdir='z')
        
        # Настройка осей
        ax.set_xlabel('Временные точки', fontsize=12, labelpad=15)
        ax.set_ylabel('Потребление воды, тонн', fontsize=12, labelpad=15)
        ax.set_zlabel('')
        ax.set_title('3D Ribbon график потребления воды', fontsize=16, pad=30)
        
        # Увеличиваем размер шрифта на осях
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10, pad=8)
        
        ax.view_init(elev=15, azim=45)
        ax.legend(fontsize=10)
        
        return fig, ax
    
    @staticmethod
    def create_3d_timeseries(dates, consumption):
        """Создает 3D временной ряд с дополнительными измерениями"""
        fig = Figure(figsize=(16, 12))
        ax = fig.add_subplot(111, projection='3d')
        
        # Преобразуем даты в числовой формат
        dates_num = mdates.date2num(dates)
        
        # Вычисляем скользящее среднее для третьего измерения
        window_size = min(7, len(consumption) // 4)
        if window_size > 1:
            moving_avg = pd.Series(consumption).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(method='ffill')
        else:
            moving_avg = consumption
        
        # Создаем 3D линию
        ax.plot(dates_num, consumption, moving_avg, 
               linewidth=3, color='purple', alpha=0.8, label='3D временной ряд')
        
        # Добавляем точки данных
        ax.scatter(dates_num, consumption, moving_avg, 
                  c=consumption, cmap='viridis', s=50, alpha=0.6)
        
        # Настройка осей
        ax.set_xlabel('Дата (числовой формат)', fontsize=12, labelpad=15)
        ax.set_ylabel('Потребление, тонн', fontsize=12, labelpad=15)
        ax.set_zlabel(f'Скользящее среднее ({window_size} дней)', fontsize=12, labelpad=15)
        ax.set_title('3D Временной ряд потребления воды', fontsize=16, pad=30)
        
        # Форматирование дат на оси X
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))
        
        # Цветовая шкала для точек
        sc = ax.scatter(dates_num, consumption, moving_avg, c=consumption, cmap='viridis', s=50, alpha=0.6)
        fig.colorbar(sc, ax=ax, shrink=0.5, aspect=5, label='Потребление, тонн')
        
        # Увеличиваем размер шрифта на осях
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10, pad=8)
        
        ax.view_init(elev=20, azim=45)
        ax.legend(fontsize=10)
        
        return fig, ax

class WaterForecastApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.users_file = "users.json"
        self.current_user = None
        self.data_file = None
        self.animation_running = False
        self.rotation_running = False
        self.rotation_angle = 0
        self.current_3d_ax = None
        self.load_users()
        self.init_ui()
        
    def load_users(self):
        try:
            with open(self.users_file, 'r') as f:
                self.users = json.load(f)
        except FileNotFoundError:
            self.users = {}
            
    def save_users(self):
        with open(self.users_file, 'w') as f:
            json.dump(self.users, f, indent=4)
            
    def register_user(self, username, password):
        if username in self.users:
            return False
        self.users[username] = password
        self.save_users()
        return True
        
    def authenticate_user(self, username, password):
        return self.users.get(username) == password
        
    def init_ui(self):
        self.setWindowTitle("Система прогнозирования потребления воды")
        self.setGeometry(100, 100, 1400, 900)
        self.setWindowIcon(WaterDropIcon.create_icon())
        
        self.setStyleSheet("""
            QMainWindow {
                background-color: #f8f9fa;
            }
            QWidget {
                font-family: "Segoe UI", Arial;
                font-size: 9pt;
            }
            QPushButton {
                background-color: #3498db;
                border: none;
                color: white;
                padding: 8px 16px;
                border-radius: 4px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #2980b9;
            }
            QPushButton:pressed {
                background-color: #21618c;
            }
            QPushButton:disabled {
                background-color: #bdc3c7;
                color: #7f8c8d;
            }
            QGroupBox {
                font-weight: bold;
                border: 2px solid #dee2e6;
                border-radius: 6px;
                margin-top: 10px;
                padding-top: 15px;
                background-color: white;
            }
            QGroupBox::title {
                subcontrol-origin: margin;
                left: 15px;
                padding: 0 8px 0 8px;
                color: #2c3e50;
            }
            QLineEdit {
                border: 1px solid #ced4da;
                padding: 6px;
                border-radius: 4px;
                background-color: white;
                font-size: 9pt;
            }
            QLineEdit:focus {
                border: 1px solid #3498db;
                background-color: #f8f9fa;
            }
            QTabWidget::pane {
                border: 1px solid #dee2e6;
                background-color: white;
                border-radius: 4px;
            }
            QTabBar::tab {
                background-color: #e9ecef;
                border: 1px solid #dee2e6;
                padding: 8px 16px;
                border-top-left-radius: 4px;
                border-top-right-radius: 4px;
                margin-right: 2px;
                color: #6c757d;
            }
            QTabBar::tab:selected {
                background-color: white;
                color: #3498db;
                border-bottom: 2px solid #3498db;
            }
            QTabBar::tab:hover {
                background-color: #dee2e6;
            }
            QTextEdit {
                border: 1px solid #ced4da;
                border-radius: 4px;
                background-color: white;
                font-size: 9pt;
                line-height: 1.4;
            }
            QTableWidget {
                border: 1px solid #dee2e6;
                border-radius: 4px;
                background-color: white;
                gridline-color: #dee2e6;
            }
            QTableWidget::item {
                padding: 6px;
                border-bottom: 1px solid #dee2e6;
            }
            QTableWidget::item:selected {
                background-color: #3498db;
                color: white;
            }
            QHeaderView::section {
                background-color: #f8f9fa;
                padding: 8px;
                border: none;
                border-bottom: 2px solid #dee2e6;
                font-weight: bold;
            }
            QProgressBar {
                border: 1px solid #ced4da;
                border-radius: 4px;
                background-color: #e9ecef;
                text-align: center;
            }
            QProgressBar::chunk {
                background-color: #3498db;
                border-radius: 3px;
            }
            QSlider::groove:horizontal {
                border: 1px solid #bbb;
                background: white;
                height: 6px;
                border-radius: 3px;
            }
            QSlider::sub-page:horizontal {
                background: #3498db;
                border: 1px solid #777;
                height: 6px;
                border-radius: 3px;
            }
            QSlider::handle:horizontal {
                background: #3498db;
                border: 1px solid #3498db;
                width: 16px;
                height: 16px;
                border-radius: 8px;
                margin: -5px 0;
            }
        """)
        
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout(central_widget)
        layout.setContentsMargins(10, 10, 10, 10)
        layout.setSpacing(10)
        
        header_layout = QHBoxLayout()
        
        title_layout = QHBoxLayout()
        icon_label = QLabel()
        icon_pixmap = WaterDropIcon.create_icon().pixmap(32, 32)
        icon_label.setPixmap(icon_pixmap)
        title_layout.addWidget(icon_label)
        
        title = QLabel("Система прогнозирования потребления воды")
        title_font = QFont()
        title_font.setPointSize(16)
        title_font.setBold(True)
        title.setFont(title_font)
        title.setStyleSheet("color: #2c3e50;")
        title_layout.addWidget(title)
        title_layout.addStretch()
        
        header_layout.addLayout(title_layout)
        
        user_info_layout = QVBoxLayout()
        self.user_label = QLabel("Пользователь: ")
        self.user_label.setStyleSheet("color: #7f8c8d; font-size: 9pt;")
        user_info_layout.addWidget(self.user_label)
        
        date_label = QLabel(datetime.now().strftime("%d.%m.%Y %H:%M"))
        date_label.setStyleSheet("color: #7f8c8d; font-size: 8pt;")
        user_info_layout.addWidget(date_label)
        
        header_layout.addLayout(user_info_layout)
        
        layout.addLayout(header_layout)
        
        self.tabs = QTabWidget()
        layout.addWidget(self.tabs)
        
        self.data_tab = QWidget()
        self.data_layout = QVBoxLayout(self.data_tab)
        self.setup_data_tab()
        self.tabs.addTab(self.data_tab, "📊 Данные")
        
        self.forecast_tab = QWidget()
        self.forecast_layout = QVBoxLayout(self.forecast_tab)
        self.setup_forecast_tab()
        self.tabs.addTab(self.forecast_tab, "🔮 Прогноз")
        
        self.visualization_tab = QWidget()
        self.visualization_layout = QVBoxLayout(self.visualization_tab)
        self.setup_visualization_tab()
        self.tabs.addTab(self.visualization_tab, "🌐 3D Анализ")
        
        self.status_bar = self.statusBar()
        self.status_bar.showMessage("Готов к работе")
        
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_time)
        self.timer.start(1000)
        
    def update_time(self):
        current_time = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
        self.status_bar.showMessage(f"Готов к работе | {current_time}")
        
    def setup_data_tab(self):
        data_group = QGroupBox("Выбор данных для анализа")
        data_form = QFormLayout()
        data_form.setVerticalSpacing(10)
        
        self.data_path_edit = QLineEdit()
        self.data_path_edit.setReadOnly(True)
        browse_btn = AnimatedButton("📁 Обзор...")
        browse_btn.clicked.connect(self.browse_data_file)
        browse_btn.setStyleSheet("padding: 6px 12px;")
        
        path_layout = QHBoxLayout()
        path_layout.addWidget(self.data_path_edit)
        path_layout.addWidget(browse_btn)
        
        data_form.addRow("Файл данных:", path_layout)
        
        format_label = QLabel("Поддерживаемые форматы: CSV, Excel. Автоматическое определение колонок")
        format_label.setStyleSheet("color: #6c757d; font-size: 8pt; font-style: italic;")
        data_form.addRow("", format_label)
        
        data_group.setLayout(data_form)
        self.data_layout.addWidget(data_group)
        
        self.progress_bar = QProgressBar()
        self.progress_bar.setVisible(False)
        self.data_layout.addWidget(self.progress_bar)
        
        table_group = QGroupBox("Просмотр данных")
        table_layout = QVBoxLayout()
        self.data_table = QTableWidget()
        self.data_table.setAlternatingRowColors(True)
        table_layout.addWidget(self.data_table)
        table_group.setLayout(table_layout)
        self.data_layout.addWidget(table_group)
        
        self.data_info_group = QGroupBox("Информация о данных")
        self.data_info_layout = QVBoxLayout()
        self.data_info_text = QTextEdit()
        self.data_info_text.setReadOnly(True)
        self.data_info_text.setMaximumHeight(180)
        self.data_info_layout.addWidget(self.data_info_text)
        self.data_info_group.setLayout(self.data_info_layout)
        self.data_layout.addWidget(self.data_info_group)
        
    def setup_forecast_tab(self):
        calc_group = QGroupBox("Прогнозирование")
        calc_layout = QHBoxLayout()
        
        self.calc_btn = AnimatedButton("📈 Рассчитать прогноз")
        self.calc_btn.clicked.connect(self.calculate_forecast)
        self.calc_btn.setEnabled(False)
        self.calc_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt;")
        
        self.details_btn = AnimatedButton("📋 Детальный анализ")
        self.details_btn.clicked.connect(self.show_detailed_analysis)
        self.details_btn.setEnabled(False)
        self.details_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt; background-color: #27ae60;")
        
        # Отключаем анимацию для кнопки анимации графика
        self.animate_btn = AnimatedButton("🎬 Анимировать график", enable_animation=False)
        self.animate_btn.clicked.connect(self.toggle_animation)
        self.animate_btn.setEnabled(False)
        self.animate_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt; background-color: #e67e22;")
        
        calc_layout.addWidget(self.calc_btn)
        calc_layout.addWidget(self.details_btn)
        calc_layout.addWidget(self.animate_btn)
        calc_layout.addStretch()
        
        calc_group.setLayout(calc_layout)
        self.forecast_layout.addWidget(calc_group)
        
        self.animation_slider_group = QGroupBox("Управление анимацией")
        self.animation_slider_layout = QVBoxLayout()
        
        self.animation_slider = QSlider(Qt.Horizontal)
        self.animation_slider.setMinimum(0)
        self.animation_slider.setMaximum(200)
        self.animation_slider.setValue(0)
        self.animation_slider.setEnabled(False)
        self.animation_slider.valueChanged.connect(self.update_animation_frame)
        
        self.slider_label = QLabel("Прогресс анимации: 0%")
        self.animation_slider_layout.addWidget(self.slider_label)
        self.animation_slider_layout.addWidget(self.animation_slider)
        
        self.animation_slider_group.setLayout(self.animation_slider_layout)
        self.animation_slider_group.setVisible(False)
        self.forecast_layout.addWidget(self.animation_slider_group)
        
        splitter = QSplitter(Qt.Vertical)
        splitter.setChildrenCollapsible(False)
        
        graph_widget = QWidget()
        graph_layout = QVBoxLayout(graph_widget)
        graph_layout.setContentsMargins(5, 5, 5, 5)
        
        graph_title = QLabel("График прогноза потребления воды")
        graph_title.setStyleSheet("font-weight: bold; color: #2c3e50; font-size: 12pt; margin: 5px;")
        graph_title.setAlignment(Qt.AlignCenter)
        graph_layout.addWidget(graph_title)
        
        self.figure = Figure(figsize=(10, 6), dpi=100)
        self.canvas = FigureCanvas(self.figure)
        self.canvas.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        graph_layout.addWidget(self.canvas)
        
        splitter.addWidget(graph_widget)
        
        results_widget = QWidget()
        results_layout = QVBoxLayout(results_widget)
        results_layout.setContentsMargins(5, 5, 5, 5)
        
        results_title = QLabel("Результаты анализа")
        results_title.setStyleSheet("font-weight: bold; color: #2c3e50; font-size: 12pt; margin: 5px;")
        results_title.setAlignment(Qt.AlignCenter)
        results_layout.addWidget(results_title)
        
        self.results_text = QTextEdit()
        self.results_text.setReadOnly(True)
        self.results_text.setMinimumHeight(200)
        results_layout.addWidget(self.results_text)
        
        splitter.addWidget(results_widget)
        
        splitter.setSizes([600, 300])
        self.forecast_layout.addWidget(splitter, 1)
        
    def setup_visualization_tab(self):
        viz_group = QGroupBox("3D визуализация данных")
        viz_layout = QVBoxLayout()
        
        viz_buttons_layout = QHBoxLayout()
        
        self.waterfall_btn = AnimatedButton("🌊 3D Waterfall")
        self.waterfall_btn.clicked.connect(lambda: self.create_3d_visualization('waterfall'))
        self.waterfall_btn.setEnabled(False)
        self.waterfall_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt; background-color: #3498db;")
        viz_buttons_layout.addWidget(self.waterfall_btn)
        
        self.surface_btn = AnimatedButton("🏔️ 3D Surface")
        self.surface_btn.clicked.connect(lambda: self.create_3d_visualization('surface'))
        self.surface_btn.setEnabled(False)
        self.surface_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt; background-color: #e67e22;")
        viz_buttons_layout.addWidget(self.surface_btn)
        
        self.ribbon_btn = AnimatedButton("🎀 3D Ribbon")
        self.ribbon_btn.clicked.connect(lambda: self.create_3d_visualization('ribbon'))
        self.ribbon_btn.setEnabled(False)
        self.ribbon_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt; background-color: #9b59b6;")
        viz_buttons_layout.addWidget(self.ribbon_btn)
        
        self.timeseries_btn = AnimatedButton("📈 3D Time Series")
        self.timeseries_btn.clicked.connect(lambda: self.create_3d_visualization('timeseries'))
        self.timeseries_btn.setEnabled(False)
        self.timeseries_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt; background-color: #27ae60;")
        viz_buttons_layout.addWidget(self.timeseries_btn)
        
        viz_buttons_layout.addStretch()
        viz_layout.addLayout(viz_buttons_layout)
        
        # Кнопки управления 3D графиком
        control_layout = QHBoxLayout()
        
        self.expand_btn = AnimatedButton("⛶ Развернуть 3D график")
        self.expand_btn.clicked.connect(self.expand_3d_visualization)
        self.expand_btn.setEnabled(False)
        self.expand_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt; background-color: #1abc9c;")
        control_layout.addWidget(self.expand_btn)
        
        self.rotate_btn = AnimatedButton("🔄 Вращать 3D график", enable_animation=False)
        self.rotate_btn.clicked.connect(self.toggle_rotation)
        self.rotate_btn.setEnabled(False)
        self.rotate_btn.setStyleSheet("padding: 10px 20px; font-size: 10pt; background-color: #e74c3c;")
        control_layout.addWidget(self.rotate_btn)
        
        control_layout.addStretch()
        viz_layout.addLayout(control_layout)
        
        # Контейнер для 3D графика
        graph_container = QWidget()
        graph_layout = QVBoxLayout(graph_container)
        graph_layout.setContentsMargins(5, 5, 5, 5)
        
        graph_title = QLabel("3D Визуализация потребления воды")
        graph_title.setStyleSheet("font-weight: bold; color: #2c3e50; font-size: 12pt; margin: 5px;")
        graph_title.setAlignment(Qt.AlignCenter)
        graph_layout.addWidget(graph_title)
        
        # Создаем область прокрутки для 3D графика
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        scroll_area.setHorizontalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        scroll_area.setVerticalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        scroll_area.setMinimumHeight(500)
        
        # Увеличиваем размер 3D фигуры
        self.figure_3d = Figure(figsize=(12, 8), dpi=100)
        self.canvas_3d = FigureCanvas(self.figure_3d)
        self.canvas_3d.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        self.canvas_3d.setMinimumSize(800, 500)
        
        scroll_area.setWidget(self.canvas_3d)
        graph_layout.addWidget(scroll_area)
        
        viz_layout.addWidget(graph_container, 1)
        
        viz_group.setLayout(viz_layout)
        self.visualization_layout.addWidget(viz_group)
        
    def expand_3d_visualization(self):
        """Открыть 3D визуализацию в отдельном окне"""
        if not hasattr(self, 'current_viz_type') or not hasattr(self, 'processed_data'):
            QMessageBox.warning(self, "Ошибка", "Сначала создайте 3D визуализацию")
            return
            
        try:
            # Подготавливаем данные для передачи в диалог
            data = {
                'dates': self.processed_data['date'],
                'consumption': self.processed_data['consumption']
            }
            
            if 'confidence_lower' in self.processed_data.columns and 'confidence_upper' in self.processed_data.columns:
                data['confidence_lower'] = self.processed_data['confidence_lower']
                data['confidence_upper'] = self.processed_data['confidence_upper']
            
            # Создаем диалог
            dialog = ThreeDVisualizationDialog(self, self.current_viz_type, data)
            dialog.exec_()
            
        except Exception as e:
            QMessageBox.critical(self, "Ошибка", f"Не удалось открыть 3D визуализацию: {str(e)}")
        
    def browse_data_file(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Выберите файл с данными", "", 
            "CSV Files (*.csv);;Excel Files (*.xlsx);;All Files (*)"
        )
        
        if file_path:
            self.data_file = file_path
            self.data_path_edit.setText(file_path)
            self.progress_bar.setVisible(True)
            self.progress_bar.setValue(0)
            QTimer.singleShot(100, self.load_and_analyze_data)

    def detect_columns(self, df):
        column_mapping = {}
        
        date_columns = ['date', 'datetime', 'time', 'дата', 'время', 'timestamp']
        for col in df.columns:
            col_lower = col.lower()
            for date_col in date_columns:
                if date_col in col_lower:
                    column_mapping['date'] = col
                    break
            if 'date' in column_mapping:
                break
        
        if 'date' not in column_mapping:
            for col in df.columns:
                try:
                    pd.to_datetime(df[col].head())
                    column_mapping['date'] = col
                    break
                except:
                    continue
        
        consumption_columns = [
            'consumption', 'water_consumption', 'predicted_water_consumption_tonnes',
            'water_consumption_tonnes_forecast', 'water_consumption_tonnes',
            'потребление', 'потребление_воды', 'forecast', 'predicted', 'value',
            'water', 'usage', 'расход'
        ]
        for col in df.columns:
            col_lower = col.lower()
            for cons_col in consumption_columns:
                if cons_col in col_lower:
                    column_mapping['consumption'] = col
                    break
            if 'consumption' in column_mapping:
                break
        
        confidence_columns = {
            'confidence_lower': ['confidence_lower', 'conf_lower', 'lower', 'нижняя', 'lower_bound'],
            'confidence_upper': ['confidence_upper', 'conf_upper', 'upper', 'верхняя', 'upper_bound']
        }
        
        for target_col, possible_names in confidence_columns.items():
            for col in df.columns:
                col_lower = col.lower()
                for name in possible_names:
                    if name in col_lower:
                        column_mapping[target_col] = col
                        break
                if target_col in column_mapping:
                    break
        
        return column_mapping
    
    def load_and_analyze_data(self):
        try:
            self.progress_bar.setValue(30)
            
            if self.data_file.endswith('.csv'):
                self.data = pd.read_csv(self.data_file)
            else:
                self.data = pd.read_excel(self.data_file)
            
            self.progress_bar.setValue(60)
            
            column_mapping = self.detect_columns(self.data)
            
            if 'date' not in column_mapping:
                QMessageBox.warning(self, "Ошибка", 
                                  "Не удалось определить колонку с датой. "
                                  f"Доступные колонки: {', '.join(self.data.columns)}")
                self.progress_bar.setVisible(False)
                return
                
            if 'consumption' not in column_mapping:
                QMessageBox.warning(self, "Ошибка", 
                                  "Не удалось определить колонку с потреблением. "
                                  f"Доступные колонки: {', '.join(self.data.columns)}")
                self.progress_bar.setVisible(False)
                return
            
            self.column_mapping = column_mapping
            
            self.processed_data = self.data.copy()
            self.processed_data = self.processed_data.rename(columns={
                column_mapping['date']: 'date',
                column_mapping['consumption']: 'consumption'
            })
            
            if 'confidence_lower' in column_mapping:
                self.processed_data = self.processed_data.rename(columns={
                    column_mapping['confidence_lower']: 'confidence_lower'
                })
                
            if 'confidence_upper' in column_mapping:
                self.processed_data = self.processed_data.rename(columns={
                    column_mapping['confidence_upper']: 'confidence_upper'
                })
            
            # Convert date column to datetime
            self.processed_data['date'] = pd.to_datetime(self.processed_data['date'])
            
            self.populate_data_table()
            self.update_data_info()
            
            self.progress_bar.setValue(100)
            QTimer.singleShot(500, lambda: self.progress_bar.setVisible(False))
            
            self.calc_btn.setEnabled(True)
            self.waterfall_btn.setEnabled(True)
            self.surface_btn.setEnabled(True)
            self.ribbon_btn.setEnabled(True)
            self.timeseries_btn.setEnabled(True)
            self.expand_btn.setEnabled(True)
            self.statusBar().showMessage(f"Данные загружены: {self.processed_data.shape[0]} записей")
            
        except Exception as e:
            self.progress_bar.setVisible(False)
            QMessageBox.critical(self, "Ошибка", f"Не удалось загрузить данные: {str(e)}")
            
    def populate_data_table(self):
        self.data_table.setRowCount(len(self.processed_data))
        self.data_table.setColumnCount(len(self.processed_data.columns))
        self.data_table.setHorizontalHeaderLabels(self.processed_data.columns)
        
        for i, row in self.processed_data.iterrows():
            for j, value in enumerate(row):
                item = QTableWidgetItem(str(value))
                self.data_table.setItem(i, j, item)
        
        self.data_table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.data_table.setAlternatingRowColors(True)
        self.data_table.setSelectionBehavior(QTableWidget.SelectRows)
        
    def update_data_info(self):
        info_text = f"📊 Файл данных: {os.path.basename(self.data_file)}\n"
        info_text += f"📏 Размер данных: {self.data.shape[0]} строк, {self.data.shape[1]} столбцов\n\n"
        
        info_text += "🔍 Обнаруженные колонки:\n"
        for standard_name, original_name in self.column_mapping.items():
            info_text += f"  • {standard_name}: {original_name}\n"
        
        info_text += f"\n📋 Структура данных:\n"
        info_text += f"Колонки: {', '.join(self.processed_data.columns)}\n\n"
        
        info_text += "📊 Типы данных:\n"
        for col in self.processed_data.columns:
            info_text += f"  • {col}: {self.processed_data[col].dtype}\n"
        
        info_text += f"\n⚠️ Пропущенные значения:\n{self.processed_data.isnull().sum()}\n"
        
        info_text += "\n📈 Статистика потребления:\n"
        info_text += f"• Минимальное потребление: {self.processed_data['consumption'].min():.2f} тонн\n"
        info_text += f"• Максимальное потребление: {self.processed_data['consumption'].max():.2f} тонн\n"
        info_text += f"• Среднее потребление: {self.processed_data['consumption'].mean():.2f} тонн\n"
        info_text += f"• Стандартное отклонение: {self.processed_data['consumption'].std():.2f} тонн"
        
        if 'confidence_lower' in self.processed_data.columns and 'confidence_upper' in self.processed_data.columns:
            info_text += "\n\n🎯 Доверительные интервалы: ДА\n"
            info_text += f"• Средняя ширина интервала: {(self.processed_data['confidence_upper'] - self.processed_data['confidence_lower']).mean():.2f} тонн"
            info_text += f"\n• Точность прогноза: {(1 - ((self.processed_data['confidence_upper'] - self.processed_data['confidence_lower']).mean() / self.processed_data['consumption'].mean())) * 100:.1f}%"
        
        self.data_info_text.setText(info_text)
            
    def calculate_forecast(self):
        if self.data_file is None:
            QMessageBox.warning(self, "Ошибка", "Сначала выберите файл с данными")
            return
            
        try:
            if not hasattr(self, 'processed_data') or self.processed_data.empty:
                QMessageBox.warning(self, "Ошибка", "Данные не загружены")
                return
                
            self.processed_data['date'] = pd.to_datetime(self.processed_data['date'])
            self.processed_data = self.processed_data.sort_values('date')
            
            self.create_enhanced_plot()
            
            report = self.generate_detailed_report()
            self.results_text.setText(report)
            
            self.details_btn.setEnabled(True)
            self.animate_btn.setEnabled(True)
            self.statusBar().showMessage("Прогноз рассчитан успешно")
            
        except Exception as e:
            QMessageBox.critical(self, "Ошибка", f"Ошибка при расчете прогноза: {str(e)}")
            
    def create_enhanced_plot(self):
        self.figure.clear()
        
        plt.style.use('default')
        
        ax = self.figure.add_subplot(111)
        
        self.figure.subplots_adjust(left=0.1, right=0.95, top=0.9, bottom=0.2)
        
        line = ax.plot(self.processed_data['date'], self.processed_data['consumption'], 
                      color='#3498db', linewidth=2.5, label='Прогнозируемое потребление', 
                      alpha=0.8, marker='o', markersize=3)
        
        if 'confidence_lower' in self.processed_data.columns and 'confidence_upper' in self.processed_data.columns:
            ax.fill_between(self.processed_data['date'], 
                          self.processed_data['confidence_lower'], 
                          self.processed_data['confidence_upper'], 
                          alpha=0.3, color='#3498db', label='Доверительный интервал (95%)')
        
        ax.set_xlabel('Дата', fontsize=12, fontweight='bold', labelpad=10)
        ax.set_ylabel('Потребление воды, тонн', fontsize=12, fontweight='bold', labelpad=10)
        ax.set_title('Прогноз потребления воды\nАнализ временного ряда', 
                    fontsize=14, fontweight='bold', pad=20)
        
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))
        ax.xaxis.set_major_locator(mdates.AutoDateLocator(maxticks=10))
        
        for tick in ax.get_xticklabels():
            tick.set_rotation(45)
            tick.set_horizontalalignment('right')
            tick.set_fontsize(10)
        
        for tick in ax.get_yticklabels():
            tick.set_fontsize(10)
        
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.set_axisbelow(True)
        
        max_consumption = self.processed_data['consumption'].max()
        min_consumption = self.processed_data['consumption'].min()
        avg_consumption = self.processed_data['consumption'].mean()
        
        max_date = self.processed_data.loc[self.processed_data['consumption'].idxmax(), 'date']
        min_date = self.processed_data.loc[self.processed_data['consumption'].idxmin(), 'date']
        
        y_range = max_consumption - min_consumption
        
        max_annotation_offset = (20, 30) if max_consumption - avg_consumption > y_range * 0.3 else (20, -30)
        min_annotation_offset = (10, -40) if avg_consumption - min_consumption > y_range * 0.3 else (10, 30)
        
        ax.annotate(f'Максимум: {max_consumption:.0f} т', 
                   xy=(max_date, max_consumption), 
                   xytext=max_annotation_offset, textcoords='offset points',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='red', alpha=0.7),
                   arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
                   fontsize=9, fontweight='bold')
        
        ax.annotate(f'Минимум: {min_consumption:.0f} т', 
                   xy=(min_date, min_consumption), 
                   xytext=min_annotation_offset, textcoords='offset points',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='green', alpha=0.7),
                   arrowprops=dict(arrowstyle='->', color='green', lw=1.5),
                   fontsize=9, fontweight='bold')
        
        ax.axhline(y=avg_consumption, color='orange', linestyle='--', alpha=0.8, 
                  label=f'Среднее: {avg_consumption:.0f} т')
        
        ax.legend(loc='upper left', frameon=True, fancybox=True, shadow=True,
                 bbox_to_anchor=(0, 1), fontsize=10, framealpha=0.9)
        
        self.figure.tight_layout(pad=3.0)
        self.canvas.draw()
        
    def toggle_animation(self):
        if not self.animation_running:
            self.start_animation()
        else:
            self.stop_animation()
            
    def start_animation(self):
        self.animation_running = True
        self.animate_btn.setText("⏹️ Остановить анимацию")
        self.animation_slider_group.setVisible(True)
        self.animation_slider.setEnabled(True)
        
        self.prepare_smooth_animation()
        
        self.animation_timer = QTimer()
        self.animation_timer.timeout.connect(self.animate_step)
        self.animation_frame = 0
        self.animation_timer.start(30)
        
    def prepare_smooth_animation(self):
        dates_num = mdates.date2num(self.processed_data['date'])
        consumption = self.processed_data['consumption']
        
        self.interp_func = interp1d(dates_num, consumption, kind='cubic', 
                                   fill_value='extrapolate')
        
        self.smooth_dates_num = np.linspace(dates_num.min(), dates_num.max(), 200)
        self.smooth_consumption = self.interp_func(self.smooth_dates_num)
        self.smooth_dates = mdates.num2date(self.smooth_dates_num)
        
    def stop_animation(self):
        self.animation_running = False
        self.animate_btn.setText("🎬 Анимировать график")
        self.animation_slider_group.setVisible(False)
        
        if hasattr(self, 'animation_timer'):
            self.animation_timer.stop()
            
    def animate_step(self):
        if self.animation_frame >= 200:
            self.stop_animation()
            return
            
        self.animation_frame += 1
        self.animation_slider.setValue(self.animation_frame)
        self.slider_label.setText(f"Прогресс анимации: {self.animation_frame/2:.1f}%")
        
        self.create_smooth_animated_plot(self.animation_frame / 200)
        
    def update_animation_frame(self, value):
        if not self.animation_running:
            self.create_smooth_animated_plot(value / 200)
            self.slider_label.setText(f"Прогресс анимации: {value/2:.1f}%")
        
    def create_smooth_animated_plot(self, progress):
        self.figure.clear()
        ax = self.figure.add_subplot(111)
        
        visible_points = int(len(self.smooth_dates) * progress)
        
        if visible_points < 2:
            visible_points = 2
            
        visible_dates = self.smooth_dates[:visible_points]
        visible_consumption = self.smooth_consumption[:visible_points]
        
        line = ax.plot(visible_dates, visible_consumption, 
                      color='#3498db', linewidth=2.5, label='Прогнозируемое потребление', 
                      alpha=0.8, marker='', markersize=0)
        
        if 'confidence_lower' in self.processed_data.columns and 'confidence_upper' in self.processed_data.columns:
            conf_lower_interp = interp1d(mdates.date2num(self.processed_data['date']), 
                                       self.processed_data['confidence_lower'], 
                                       kind='cubic', fill_value='extrapolate')
            conf_upper_interp = interp1d(mdates.date2num(self.processed_data['date']), 
                                       self.processed_data['confidence_upper'], 
                                       kind='cubic', fill_value='extrapolate')
            
            smooth_conf_lower = conf_lower_interp(self.smooth_dates_num[:visible_points])
            smooth_conf_upper = conf_upper_interp(self.smooth_dates_num[:visible_points])
            
            ax.fill_between(visible_dates, smooth_conf_lower, smooth_conf_upper, 
                          alpha=0.3, color='#3498db', label='Доверительный интервал (95%)')
        
        ax.set_xlabel('Дата', fontsize=12, fontweight='bold', labelpad=10)
        ax.set_ylabel('Потребление воды, тонн', fontsize=12, fontweight='bold', labelpad=10)
        ax.set_title(f'Прогноз потребления воды (анимация: {progress*100:.1f}%)', 
                    fontsize=14, fontweight='bold', pad=20)
        
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))
        ax.xaxis.set_major_locator(mdates.AutoDateLocator(maxticks=10))
        
        for tick in ax.get_xticklabels():
            tick.set_rotation(45)
            tick.set_horizontalalignment('right')
            tick.set_fontsize(10)
        
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.set_axisbelow(True)
        
        ax.legend(loc='upper left', frameon=True, fancybox=True, shadow=True,
                 fontsize=10, framealpha=0.9)
        
        self.figure.tight_layout(pad=3.0)
        self.canvas.draw()
        
    def create_3d_visualization(self, viz_type):
        """Создает различные типы 3D визуализаций"""
        if not hasattr(self, 'processed_data') or self.processed_data.empty:
            QMessageBox.warning(self, "Ошибка", "Сначала загрузите данные")
            return
            
        try:
            # Сохраняем тип визуализации для кнопки развертывания
            self.current_viz_type = viz_type
            
            # Подготовка данных
            dates = self.processed_data['date']
            consumption = self.processed_data['consumption']
            
            confidence_lower = None
            confidence_upper = None
            if 'confidence_lower' in self.processed_data.columns and 'confidence_upper' in self.processed_data.columns:
                confidence_lower = self.processed_data['confidence_lower']
                confidence_upper = self.processed_data['confidence_upper']
            
            # Очищаем предыдущий график
            self.figure_3d.clear()
            
            # Создание выбранного типа визуализации
            if viz_type == 'waterfall':
                fig, ax = Water3DVisualizer.create_3d_waterfall(dates, consumption, confidence_lower, confidence_upper)
            elif viz_type == 'surface':
                fig, ax = Water3DVisualizer.create_3d_surface(dates, consumption)
            elif viz_type == 'ribbon':
                fig, ax = Water3DVisualizer.create_3d_ribbon(dates, consumption, confidence_lower, confidence_upper)
            elif viz_type == 'timeseries':
                fig, ax = Water3DVisualizer.create_3d_timeseries(dates, consumption)
            else:
                QMessageBox.warning(self, "Ошибка", "Неизвестный тип визуализации")
                return
            
            # Обновление canvas
            self.figure_3d = fig
            self.canvas_3d.figure = fig
            self.current_3d_ax = ax
            self.canvas_3d.draw()
            
            self.rotate_btn.setEnabled(True)
            self.statusBar().showMessage(f"3D {viz_type} визуализация построена успешно")
            
        except Exception as e:
            QMessageBox.critical(self, "Ошибка", 
                               f"Ошибка при построении 3D графика: {str(e)}\n"
                               f"Тип ошибки: {type(e).__name__}")
            
    def toggle_rotation(self):
        if not self.rotation_running:
            self.start_rotation()
        else:
            self.stop_rotation()
            
    def start_rotation(self):
        self.rotation_running = True
        self.rotate_btn.setText("⏹️ Остановить вращение")
        self.rotation_angle = 0
        
        self.rotation_timer = QTimer()
        self.rotation_timer.timeout.connect(self.rotate_3d)
        self.rotation_timer.start(50)
        
    def stop_rotation(self):
        self.rotation_running = False
        self.rotate_btn.setText("🔄 Вращать 3D график")
        
        if hasattr(self, 'rotation_timer'):
            self.rotation_timer.stop()
            
    def rotate_3d(self):
        if not hasattr(self, 'current_3d_ax') or self.current_3d_ax is None:
            return
            
        self.rotation_angle = (self.rotation_angle + 1) % 360
        
        # Плавное изменение угла обзора
        elevation = 15 + 10 * np.sin(np.radians(self.rotation_angle * 2))
        
        self.current_3d_ax.view_init(elev=elevation, azim=self.rotation_angle)
        self.canvas_3d.draw()
        
    def generate_detailed_report(self):
        report = "📊 АНАЛИЗ ПРОГНОЗА ПОТРЕБЛЕНИЯ ВОДЫ\n"
        report += "=" * 60 + "\n\n"
        
        current_consumption = self.processed_data['consumption'].iloc[0]
        final_consumption = self.processed_data['consumption'].iloc[-1]
        max_consumption = self.processed_data['consumption'].max()
        min_consumption = self.processed_data['consumption'].min()
        avg_consumption = self.processed_data['consumption'].mean()
        total_consumption = self.processed_data['consumption'].sum()
        std_consumption = self.processed_data['consumption'].std()
        cv_consumption = (std_consumption / avg_consumption) * 100
        
        max_date = self.processed_data.loc[self.processed_data['consumption'].idxmax(), 'date']
        min_date = self.processed_data.loc[self.processed_data['consumption'].idxmin(), 'date']
        
        report += "📈 ОСНОВНАЯ СТАТИСТИКА:\n"
        report += "-" * 40 + "\n"
        report += f"• Период анализа: {self.processed_data['date'].iloc[0].strftime('%d.%m.%Y')} - {self.processed_data['date'].iloc[-1].strftime('%d.%m.%Y')}\n"
        report += f"• Начальное потребление: {current_consumption:.0f} тонн/день\n"
        report += f"• Конечное потребление: {final_consumption:.0f} тонн/день\n"
        report += f"• Среднее потребление: {avg_consumption:.0f} ± {std_consumption:.0f} тонн/день\n"
        report += f"• Максимальное потребление: {max_consumption:.0f} тонн/день ({max_date.strftime('%d.%m.%Y')})\n"
        report += f"• Минимальное потребление: {min_consumption:.0f} тонн/день ({min_date.strftime('%d.%m.%Y')})\n"
        report += f"• Общее потребление за период: {total_consumption:,.0f} тонн\n"
        report += f"• Коэффициент вариации: {cv_consumption:.1f}%\n\n"
        
        trend = final_consumption - current_consumption
        trend_percent = (trend / current_consumption) * 100 if current_consumption != 0 else 0
        
        report += "📊 АНАЛИЗ ТРЕНДА:\n"
        report += "-" * 40 + "\n"
        if trend > 0:
            report += f"• 📈 ВОСХОДЯЩИЙ ТРЕНД: +{trend:.0f} тонн ({trend_percent:+.1f}%)\n"
        elif trend < 0:
            report += f"• 📉 НИСХОДЯЩИЙ ТРЕНД: {trend:.0f} тонн ({trend_percent:+.1f}%)\n"
        else:
            report += "• ➡️ СТАБИЛЬНЫЙ ТРЕНД: без значительных изменений\n"
        
        consumption_range = max_consumption - min_consumption
        variability_percent = (consumption_range / avg_consumption) * 100
        
        report += f"• Диапазон колебаний: {consumption_range:.0f} тонн ({variability_percent:.1f}% от среднего)\n\n"
        
        report += "💡 РЕКОМЕНДАЦИИ:\n"
        report += "-" * 40 + "\n"
        
        if variability_percent > 50:
            report += "• 🚨 ВЫСОКАЯ СЕЗОННАЯ ВАРИАТИВНОСТЬ\n"
            report += "  - Подготовить систему к пиковым нагрузкам\n"
            report += "  - Оптимизировать ресурсы в периоды спада\n"
            report += "  - Рассмотреть возможность создания резервных мощностей\n"
            report += "  - Внедрить систему мониторинга в реальном времени\n"
        elif variability_percent > 25:
            report += "• ⚠️ СРЕДНЯЯ СЕЗОННАЯ ВАРИАТИВНОСТЬ\n"
            report += "  - Планировать техническое обслуживание в периоды низкого потребления\n"
            report += "  - Обеспечить равномерную загрузку оборудования\n"
            report += "  - Мониторить тенденции потребления\n"
        else:
            report += "• ✅ СТАБИЛЬНОЕ ПОТРЕБЛЕНИЕ\n"
            report += "  - Поддерживать текущую стратегию управления\n"
            report += "  - Фокусироваться на энергоэффективности\n"
            report += "  - Оптимизировать операционные расходы\n"
        
        if trend_percent > 10:
            report += "• 📈 РОСТ ПОТРЕБЛЕНИЯ\n"
            report += "  - Оценить необходимость расширения мощностей\n"
            report += "  - Проанализировать причины роста\n"
        elif trend_percent < -10:
            report += "• 📉 СНИЖЕНИЕ ПОТРЕБЛЕНИЯ\n"
            report += "  - Исследовать причины снижения\n"
            report += "  - Оптимизировать операционные затраты\n"
        
        report += "\n🔧 ОПЕРАТИВНЫЕ МЕРОПРИЯТИЯ:\n"
        report += "-" * 40 + "\n"
        report += "• Ежедневный мониторинг показателей\n"
        report += "• Плановые проверки оборудования\n"
        report += "• Анализ эффективности работы системы\n"
        report += "• Подготовка отчетов для руководства\n"
        report += "• Обновление моделей прогнозирования\n"
        
        self.analysis_data = {
            'stats': {
                'period': f"{self.processed_data['date'].iloc[0].strftime('%d.%m.%Y')} - {self.processed_data['date'].iloc[-1].strftime('%d.%m.%Y')}",
                'start_cons': f"{current_consumption:.0f} тонн/день",
                'end_cons': f"{final_consumption:.0f} тонн/день",
                'avg_cons': f"{avg_consumption:.0f} ± {std_consumption:.0f} тонн/день",
                'max_cons': f"{max_consumption:.0f} тонн/день",
                'min_cons': f"{min_consumption:.0f} тонн/день",
                'total_cons': f"{total_consumption:,.0f} тонн",
                'std_dev': f"{std_consumption:.0f} тонн",
                'var_coef': f"{cv_consumption:.1f}%"
            },
            'seasonal': {},
            'recommendations': report,
            'forecast': self.generate_forecast_text(trend, trend_percent)
        }
        
        return report

    def generate_forecast_text(self, trend, trend_percent):
        forecast_text = "🔮 ПРОГНОЗ НА БЛИЖАЙШИЙ ПЕРИОД:\n\n"
        
        if trend > 0:
            forecast_text += f"• Ожидается рост потребления на {trend_percent:+.1f}%\n"
            forecast_text += "• Рекомендуется подготовить дополнительные мощности\n"
            forecast_text += "• Увеличить мониторинг качества воды\n"
        elif trend < 0:
            forecast_text += f"• Ожидается снижение потребления на {trend_percent:+.1f}%\n"
            forecast_text += "• Возможность проведения планового обслуживания\n"
            forecast_text += "• Оптимизация энергопотребления\n"
        else:
            forecast_text += "• Ожидается стабильное потребление\n"
            forecast_text += "• Поддерживать текущие операционные показатели\n"
        
        forecast_text += "\n📅 РЕКОМЕНДУЕМЫЕ МЕРОПРИЯТИЯ:\n"
        forecast_text += "• Еженедельный анализ тенденций\n"
        forecast_text += "• Корректировка моделей прогнозирования\n"
        forecast_text += "• Взаимодействие с потребителями\n"
        forecast_text += "• Подготовка сезонных планов\n"
        
        return forecast_text
    
    def show_detailed_analysis(self):
        if not hasattr(self, 'analysis_data'):
            QMessageBox.warning(self, "Ошибка", "Сначала рассчитайте прогноз")
            return
            
        self.analysis_window = AnalysisResultsWindow(self)
        self.analysis_window.set_analysis_data(
            self.analysis_data['stats'],
            self.analysis_data['seasonal'],
            self.analysis_data['recommendations'],
            self.analysis_data['forecast']
        )
        self.analysis_window.exec_()
        
    def show_main_window(self):
        self.user_label.setText(f"Пользователь: {self.current_user}")
        
        self.animation = QPropertyAnimation(self, b"windowOpacity")
        self.animation.setDuration(500)
        self.animation.setStartValue(0)
        self.animation.setEndValue(1)
        self.animation.start()
        
        self.show()

def main():
    app = QApplication(sys.argv)
    
    app.setStyle('Fusion')
    
    main_app = WaterForecastApp()
    
    login_window = LoginWindow(main_app)
    login_window.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

C:\Users\user\AppData\Local\Temp\ipykernel_12032\2840275791.py:1994: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  self.figure.tight_layout(pad=3.0)
C:\Users\user\AppData\Local\Temp\ipykernel_12032\2840275791.py:1214: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  moving_avg = pd.Series(consumption).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(method='ffill')
C:\Users\user\AppData\Local\Temp\ipykernel_12032\2840275791.py:553: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  moving_avg = pd.Series(consumption).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(method='ffill')


SystemExit: 0

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
